# Data Wrangling

### Ingest and join data

In [2]:
import pandas as pd
def import_data(name):
    df = pd.read_csv(name)
    return df

transacciones = import_data('../Algoselling transacciones.csv')
transacciones['INVOICE_DATE'] = pd.to_datetime(transacciones['INVOICE_DATE'], format='%Y%m%d')

atributos = import_data('../Algoselling atributos.csv')

data = pd.merge(transacciones, atributos, left_on='ACCOUNT_ID', right_on='POC', how='left')
data = data.loc[:,~data.columns.duplicated()]

data = data[['ACCOUNT_ID', 'SKU_ID', 'INVOICE_DATE', 'ITEMS_PHYS_CASES', 'nse']]

data.head()


,ACCOUNT_ID,SKU_ID,INVOICE_DATE,ITEMS_PHYS_CASES,nse
0,430606,7038,2022-07-29,100.0,Bajo
1,323267,14933,2022-07-29,1.0,Medio
2,357825,21971,2022-07-23,8.0,Medio
3,444926,7038,2022-08-05,20.0,Medio
4,450771,7030,2022-08-16,5.0,Bajo


### Create missing features

In [3]:
data['last_time_bought'] = data.groupby(['ACCOUNT_ID', 'SKU_ID'])['INVOICE_DATE'].shift(1)
data['last_transaction_date'] = data.groupby('ACCOUNT_ID')['INVOICE_DATE'].shift(1)
data['has_bought_before'] = data['last_time_bought'].notna()

data.head()

,ACCOUNT_ID,SKU_ID,INVOICE_DATE,ITEMS_PHYS_CASES,nse,last_time_bought,last_transaction_date,has_bought_before
0,430606,7038,2022-07-29,100.0,Bajo,NaT,NaT,False
1,323267,14933,2022-07-29,1.0,Medio,NaT,NaT,False
2,357825,21971,2022-07-23,8.0,Medio,NaT,NaT,False
3,444926,7038,2022-08-05,20.0,Medio,NaT,NaT,False
4,450771,7030,2022-08-16,5.0,Bajo,NaT,NaT,False


In [4]:
N_DAYS = 7

def calculate_last_n_days_trans(row, df, n_days):
    """
    Calculates the sum of ITEMS_PHYS_CASES for transactions up to N_DAYS before the INVOICE_DATE
    for the same ACCOUNT_ID.
    """
    cutoff_date = row['INVOICE_DATE'] - pd.Timedelta(days=n_days)
    filtered_df = df[(df['INVOICE_DATE'] >= cutoff_date) & 
                     (df['INVOICE_DATE'] < row['INVOICE_DATE'])]
    return filtered_df['ITEMS_PHYS_CASES'].sum()

def calculate_last_n_days_trans_by_sku(row, df, n_days):
    """
    Calculates the sum of ITEMS_PHYS_CASES for transactions up to N_DAYS before the INVOICE_DATE
    for the same ACCOUNT_ID and SKU_ID.
    """
    cutoff_date = row['INVOICE_DATE'] - pd.Timedelta(days=n_days)
    filtered_df = df[(df['SKU_ID'] == row['SKU_ID']) &
                     (df['INVOICE_DATE'] >= cutoff_date) & 
                     (df['INVOICE_DATE'] < row['INVOICE_DATE'])]
    return filtered_df['ITEMS_PHYS_CASES'].sum()

data['INVOICE_DATE'] = pd.to_datetime(data['INVOICE_DATE'])

data.sort_values(by='INVOICE_DATE', inplace=True)
data['LAST_N_DAYS_TRANS'] = data.apply(calculate_last_n_days_trans, df=data, n_days=N_DAYS, axis=1)

data.sort_values(by=['INVOICE_DATE', 'SKU_ID'], inplace=True)
data['LAST_N_DAYS_TRANS_by_SKU'] = data.apply(calculate_last_n_days_trans_by_sku, df=data, n_days=N_DAYS, axis=1)

data['importancia'] = data['LAST_N_DAYS_TRANS_by_SKU'] / data['LAST_N_DAYS_TRANS']

data.head()

,ACCOUNT_ID,SKU_ID,INVOICE_DATE,ITEMS_PHYS_CASES,nse,last_time_bought,last_transaction_date,has_bought_before,LAST_N_DAYS_TRANS,LAST_N_DAYS_TRANS_by_SKU,importancia
160545,453972,691,2022-05-24,2.0,S/D,2022-07-30,2022-06-11,True,0.0,0.0,NaN
38024,140565,691,2022-05-24,1.0,Medio,NaT,2022-07-23,False,0.0,0.0,NaN
87172,391399,1473,2022-05-24,2.0,Medio,NaT,2022-06-30,False,0.0,0.0,NaN
270689,165294,1473,2022-05-24,1.0,Medio,2022-08-05,2022-08-23,True,0.0,0.0,NaN
126722,453972,1483,2022-05-24,3.0,S/D,2022-06-22,2022-08-13,True,0.0,0.0,NaN


### Filtering

In [9]:
data = data[data['nse'].notna()]
data = data[data['nse'] != 'S/D']
data = data[data['last_transaction_date'].notna()]
data = data[data['importancia'].notna()]
data = data[data['importancia'] > 0]
data = data[['ACCOUNT_ID', 'SKU_ID', 'INVOICE_DATE', 'ITEMS_PHYS_CASES', 'nse', 'has_bought_before', 'importancia']]

data.head()

,ACCOUNT_ID,SKU_ID,INVOICE_DATE,ITEMS_PHYS_CASES,nse,has_bought_before,importancia
233617,408063,1483,2022-05-30,1.0,Medio,True,0.010941
242212,457424,2201,2022-05-30,1.0,Medio,True,0.015317
87232,437463,2201,2022-05-30,2.0,Medio,True,0.015317
54031,388336,2201,2022-05-30,1.0,Medio,False,0.015317
229529,427558,2201,2022-05-30,1.0,Medio,True,0.015317


### Add target variable

In [10]:
from datetime import timedelta

data['INVOICE_DATE'] = pd.to_datetime(data['INVOICE_DATE'])
data.sort_values(['ACCOUNT_ID', 'SKU_ID', 'INVOICE_DATE'], inplace=True)

def will_buy(row):
    future_transactions = data[(data['ACCOUNT_ID'] == row['ACCOUNT_ID']) &
                               (data['SKU_ID'] == row['SKU_ID']) &
                               (data['INVOICE_DATE'] > row['INVOICE_DATE'])]
    
    if not future_transactions.empty:
        next_row = future_transactions.iloc[0]
        return next_row['INVOICE_DATE'] <= row['INVOICE_DATE'] + timedelta(days=30)
    return False

data['will_buy'] = data.apply(will_buy, axis=1)

data.head()


,ACCOUNT_ID,SKU_ID,INVOICE_DATE,ITEMS_PHYS_CASES,nse,has_bought_before,importancia,will_buy
189324,22204,7038,2022-06-28,1.0,Medio,False,0.060523,True
203354,22204,7038,2022-07-12,1.0,Medio,True,0.119798,False
199549,22204,25119,2022-06-28,1.0,Medio,False,0.008374,False
267254,22247,1416,2022-07-26,1.0,Medio,False,0.008966,False
65800,22247,6582,2022-07-26,1.0,Medio,True,0.003531,True


### Feature enconding

In [11]:
data['nse'] = data['nse'].map({'Alto': 1, 'Medio': 2, 'Bajo': 3})
data['has_bought_before'] = data['has_bought_before'].astype(int)
data.head()

,ACCOUNT_ID,SKU_ID,INVOICE_DATE,ITEMS_PHYS_CASES,nse,has_bought_before,importancia,will_buy
189324,22204,7038,2022-06-28,1.0,2,0,0.060523,True
203354,22204,7038,2022-07-12,1.0,2,1,0.119798,False
199549,22204,25119,2022-06-28,1.0,2,0,0.008374,False
267254,22247,1416,2022-07-26,1.0,2,0,0.008966,False
65800,22247,6582,2022-07-26,1.0,2,1,0.003531,True


### Export data

In [38]:
data.to_csv('wrangled_data.csv', index=False)

# Model Development

### (To run in Colab)

In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
df = pd.read_csv('/content/drive/My Drive/wrangled_data.csv')
data_df = df

### Data ingestion and preparation

In [12]:
import pandas as pd
data_df = pd.read_csv('../wrangled_data.csv')
data_df.head()

,ACCOUNT_ID,SKU_ID,INVOICE_DATE,ITEMS_PHYS_CASES,nse,has_bought_before,importancia,will_buy
0,22204,7038,2022-06-28,1.0,2,0,0.060523,True
1,22204,7038,2022-07-12,1.0,2,1,0.119798,False
2,22204,25119,2022-06-28,1.0,2,0,0.008374,False
3,22204,76503,2022-06-14,1.0,2,0,0.000000,False
4,22247,1416,2022-07-26,1.0,2,0,0.008966,False


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Select features and target
features = data_df[['nse', 'has_bought_before', 'importancia']]
target = data_df['will_buy']

# Scale features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

/Users/foxfrancisco/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## A. Logistic Regression Model

### Elección del Modelo

La elección del modelo de regresión logística se debió a:

- Naturaleza de los Datos y la Tarea: Necesidad de recomendar productos basados en características de transacciones específicas.
- Presencia de Muestras Positivas Únicamente: Limitación inicial de solo contar con ejemplos de compras realizadas.
- Objetivo de Clasificación para Rankear Productos: Utilizar un modelo que permita ordenar productos según la probabilidad de compra.
- Simplicidad e Interpretabilidad del Modelo: La regresión logística es simple, interpretable y capaz de producir probabilidades adecuadas para el rankeo.
- Adaptabilidad al Problema y Factibilidad de Implementación: Capacidad de adaptar un enfoque de clasificación para servir a un propósito de rankeo, junto con una implementación y evaluación directas.

Se eligió entonces, como promera aproximación, un modelo simple de Logistic Regression con 3 features que permita manipular los datos de manera cómoda, asegurando resultados positivos y evitando fenómenos indeseables (eg: overfitting) que se asocian generalmente a modelos más complejos.

### Desarrollo

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, f1_score


# Train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict probabilities on the testing set
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Get the probability of the positive class (purchase)

# Convert probabilities to binary outcomes based on a threshold
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_proba]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")


Accuracy: 0.7183235192241081
Precision: 0.6776137856425315
F1 Score: 0.7419723315141515


Accuracy: 0.7183235192241081

Precision: 0.6776137856425315

F1 Score: 0.7419723315141515

### Selección y Evaluación de Métricas

Evaluar un modelo de clasificación requiere múltiples métricas porque una sola no puede capturar la complejidad del rendimiento del modelo. Cada métrica aporta una perspectiva distinta sobre cómo el modelo maneja diferentes aspectos de los datos.

- Accuracy (Exactitud): Mide la proporción de predicciones correctas entre el total de casos. Es útil para una visión general, pero puede ser engañosa en datos desbalanceados.
- Precision (Precisión): Indica la proporción de verdaderos positivos entre todas las predicciones positivas. Es crucial cuando los falsos positivos son particularmente problemáticos.
- F1 Score: Combina precisión y sensibilidad (recall) para medir la eficiencia del modelo en encontrar un balance entre identificar correctamente las clases positivas y evitar falsos positivos.

Mejoras basadas en las métricas:
- Para mejorar la Accuracy: Considerar ajustar los hiperparámetros, explorar diferentes modelos, o modificar el conjunto de características.
- Para aumentar la Precision: Ajustar el umbral de decisión para ser más conservador al predecir clases positivas o aplicar técnicas para equilibrar las clases.
- Para optimizar el F1 Score: Buscar un equilibrio entre precisión y sensibilidad, lo cual puede implicar ajustar el umbral de clasificación o probar modelos que capturen mejor la complejidad de los datos.

Estas métricas, juntas, ofrecen una guía completa para afinar el modelo, asegurando que se maneje adecuadamente la variedad de desafíos presentados por los datos y el contexto específico de su aplicación.

## B. NN Model

Se optó por probar un modelo simple de redes neuronales (NN) además de Logistic Regression para explorar la capacidad de las NN de capturar relaciones complejas y no lineales en los datos, ofreciendo potencialmente un mejor rendimiento en la tarea de clasificación. Comparar ambos modelos permite identificar cuál es más efectivo para predecir la probabilidad de compra, basándose en métricas de evaluación específicas y considerando la naturaleza del problema.

El modelo elegido consta de 3 layers:
- Dense Layer: 128 neuronas, activación ReLU.
- Dropout: Descarta el 20% de las neuronas al azar para prevenir el sobreajuste.
- Dense Outout Layer: 1 neurona, activación sigmoid (produce la probabilidad de clasificación binaria).

In [ ]:
import tensorflow as tf
from sklearn.metrics import precision_score, f1_score

# Asegúrate de tener el conjunto de datos preparado (X_train, X_test, y_train, y_test)

# Construyendo el modelo de red neuronal
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Capa de salida para clasificación binaria
])

# Compilando el modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Entrenando el modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=10, validation_split=0.2, verbose=1)

# Evaluando el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
y_pred_proba_nn = model.predict(X_test).ravel()  # Obtener probabilidades
y_pred_nn = [1 if prob > 0.5 else 0 for prob in y_pred_proba_nn]  # Convertir a clases binarias

# Calculando métricas adicionales
precision_nn = precision_score(y_test, y_pred_nn)
f1_nn = f1_score(y_test, y_pred_nn)

# Imprimiendo las métricas
print(f"Accuracy (NN): {accuracy}")
print(f"Precision (NN): {precision_nn}")
print(f"F1 Score (NN): {f1_nn}")


Epoch 1/10

13858/13858 [==============================] - 35s 2ms/step - loss: 0.5751 - accuracy: 0.7201 - val_loss: 0.5723 - val_accuracy: 0.7205

Epoch 2/10

13858/13858 [==============================] - 30s 2ms/step - loss: 0.5727 - accuracy: 0.7204 - val_loss: 0.5752 - val_accuracy: 0.7206

Epoch 3/10

13858/13858 [==============================] - 30s 2ms/step - loss: 0.5724 - accuracy: 0.7209 - val_loss: 0.5727 - val_accuracy: 0.7217

Epoch 4/10

13858/13858 [==============================] - 29s 2ms/step - loss: 0.5723 - accuracy: 0.7208 - val_loss: 0.5716 - val_accuracy: 0.7218

Epoch 5/10

13858/13858 [==============================] - 31s 2ms/step - loss: 0.5722 - accuracy: 0.7208 - val_loss: 0.5726 - val_accuracy: 0.7220

Epoch 6/10

13858/13858 [==============================] - 29s 2ms/step - loss: 0.5724 - accuracy: 0.7206 - val_loss: 0.5726 - val_accuracy: 0.7223

Epoch 7/10

13858/13858 [==============================] - 30s 2ms/step - loss: 0.5722 - accuracy: 0.7207 - val_loss: 0.5721 - val_accuracy: 0.7214

Epoch 8/10

13858/13858 [==============================] - 30s 2ms/step - loss: 0.5720 - accuracy: 0.7211 - val_loss: 0.5722 - val_accuracy: 0.7223

Epoch 9/10

13858/13858 [==============================] - 31s 2ms/step - loss: 0.5722 - accuracy: 0.7208 - val_loss: 0.5717 - val_accuracy: 0.7217

Epoch 10/10

13858/13858 [==============================] - 30s 2ms/step - loss: 0.5724 - accuracy: 0.7207 - val_loss: 0.5717 - val_accuracy: 0.7215

1354/1354 [==============================] - 2s 1ms/step - loss: 0.5739 - accuracy: 0.7177

1354/1354 [==============================] - 2s 1ms/step


Accuracy (NN): 0.7177231311798096

Precision (NN): 0.6785742111414103

F1 Score (NN): 0.7402575326165484